In [ ]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import matplotlib.gridspec as gridspec
from PIL import Image

In [ ]:
classes=[]
for fold_name in np.sort(os.listdir('/content/drive/MyDrive/DL_DATA_iNat/val')):
  classes.append(fold_name)
print(classes)

In [ ]:
print(os.listdir('/content/drive/MyDrive/DL_DATA_iNat/val/Aves'))

In [ ]:
@tf.custom_gradient
def guidedRelu(x):
  def grad(dy):
    return tf.cast(dy>0,"float32") * tf.cast(x>0, "float32") * dy
  return tf.nn.relu(x), grad

In [ ]:
import keras.backend as kb

def deprocess_image(img):
    img = img.copy()
    img = img - img.mean()
    img /= (img.std() + K.epsilon())
    img = img * 0.25

    img += 0.5
    img = np.clip(img, 0, 1)

    img *= 255
    if kb.image_data_format() == 'channels_first':
        img = img.transpose((1, 2, 0))
    img = np.clip(img, 0, 255).astype('uint8')

    return img


In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/DL_DATA_iNat/model-best.h5/')
gd_bp_model = Model(inputs = [model.inputs], outputs = [model.get_layer("conv2d_4").output])

for layer in model.layers:
  if hasattr(layer, 'activation') and layer.activation==tf.keras.activations.relu:
    layer.activation = guidedRelu

In [ ]:
import random

path = '/content/drive/MyDrive/DL_DATA_iNat/val/'

Class_num = random.randint(0,9)
files = os.listdir(path+classes[Class_num])

file_num = random.randint(0,len(files)-1)
print("Class num = " + str(Class_num) + " "+classes[Class_num])
plt.imshow(Image.open(path+classes[Class_num]+'/'+files[file_num]))
plt.title(classes[Class_num])

In [ ]:
fin_layer = model.get_layer("conv2d_4").output.shape[1:]

test_img = Image.open(path+classes[Class_num]+'/'+files[file_num])
test_img = test_img.resize((227,227))
t_img_arr = np.asarray(test_img)*1./255

guid_backprops = []
numOfNeurons = 10
for i in range(numOfNeurons):
  neuron = [np.random.randint(0,d-1) for d in fin_layer]
  mask = np.zeros(fin_layer)
  mask[neuron[0],neuron[1],neuron[2]] = 1
  with tf.GradientTape() as tape:
    inps = tf.cast([t_img_arr], tf.float32)
    tape.watch(inps)
    ops = gd_bp_model(inps) * mask

  guid_backprops.append((neuron,tape.gradient(ops,inps)[0]))

In [ ]:
fig,ax = plt.subplots(numOfNeurons,1,figsize=(20,40))

for i,(neuron,image) in enumerate(guid_backprops):
    ax[i].imshow(deprocess_image(np.array(image)))
    ax[i].tick_params(axis='both',which='both',labelbottom=False,labelleft=False,labeltop=False)
    ax[i].axis("off")
    ax[i].set_title("Neuron: {}".format(neuron))
    
plt.tight_layout()
plt.savefig('PA-Q5-2.png',bbox_inches='tight')